# Compare sequential design strategies.

In [2]:
import GPy
import numpy as np

We'll use the branin function as a test function, whcih has domain x1 ∈ [-5, 10], x2 ∈ [0, 15].

In [3]:
def branin(x):
    if(x.ndim==1):
        x=x.reshape((1,2))
    x1 = x[:,0]
    x2 = x[:,1]
    a = 1.
    b = 5.1 / (4.*np.pi**2)
    c = 5. / np.pi
    r = 6.
    s = 10.
    t = 1. / (8.*np.pi)
    ret  = a*(x2-b*x1**2+c*x1-r)**2+s*(1-t)*np.cos(x1)+s
    return ret[:,None]

In [4]:
x1,x2 = np.meshgrid(np.linspace(-5,10,100), np.linspace(0,15,100))
Xfinegrid=np.concatenate((x1.flatten()[:,None],x2.flatten()[:,None]),axis=1)
Yfinegrid = branin(Xfinegrid)

In [5]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

data = [
    go.Contour(
        z=Yfinegrid.reshape((100,100)),
        x=np.linspace(-5,10,100),
        y=np.linspace(0,15,100),
        contours = dict(
            start = 0,
            end = 300,
            size = 10
        ),
    )]

plotly.offline.iplot(data)




The first method adds new design points at places where the variance in the GP prediction is largest. To get a fair reflection of the method, we repeat the process multiple times from different starting points and then average.

In [6]:
def AddPoint_Method1(m, Xref):
    # m the current GP
    # Xref the set of candidate points
    # return updated model, index of training point added, new training set and new reference set
    mu,var = m._raw_predict(Xref)
    index = var.argmax()
    Xtrain_new = np.concatenate((m.X, Xref[index,:][None,:]),axis=0)
    ytrain_new = np.append(m.Y, branin(Xtrain_new[-1,:]))[:,None]
    Xref_new = np.delete(Xref, index, axis=0)# remove index row
    m.set_XY(Xtrain_new, ytrain_new)
    return(index, m, Xtrain_new, ytrain_new, Xref_new)

def AssessMethod1(nMax, nMC):
    RMSE_1 = np.zeros((nMax,2,nMC))
    for mm in range(nMC):
        print('---------------------------------')
        index0 = np.random.randint(Xfinegrid.shape[0])
        X0 = Xfinegrid[index0,:].reshape((1,2))
        Xref = np.delete(Xfinegrid,index0, axis=0)
        ytrain=branin(X0)
        Xtrain = X0

        k = GPy.kern.Matern52(2, ARD=True)#GPy.kern.Bias(2)+GPy.
        m = GPy.models.GPRegression(Xtrain, ytrain,k)
        m.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
        m.optimize_restarts(1, robust=True, verbose=False)

        
        for ii in range(nMax):
            index, m, Xtrain, ytrain, Xref = AddPoint_Method1(m, Xref)
            # For some reason updating the m returned here doesn't work.
            # So I'm just refitting the model
            k = GPy.kern.Matern52(2)#+GPy.kern.Bias(2)
            m = GPy.models.GPRegression(Xtrain, ytrain,k)
            m.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
        
        #    m.Gaussian_noise.constrain_bounded(0,10)
            m.optimize_restarts(3, robust=True, verbose=False)
            #print(m)
            mu, _ = m._raw_predict(Xref)
            RMSE_1[ii,1,mm] = np.sqrt(np.mean((mu - branin(Xref))**2))
            RMSE_1[ii,0,mm] = ii   
        
    #print(RMSE_1)
    return(RMSE_1)

out1 =AssessMethod1(40,10)

---------------------------------


/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

overflow encountered in square

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

invalid value encountered in multiply

/Users/pmzrdw/anaconda/lib/python3.5/site-packages/paramz/transformations.py:106: RuntimeWarning:

invalid value encountered in greater

/Users/pmzrdw/anaconda/lib/python3.5/site-packages/paramz/transformations.py:111: RuntimeWarning:

invalid value encountered in greater

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:160: RuntimeWarning:

overflow encountered in true_divide

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

overflow encountered in multiply



---------------------------------
---------------------------------
---------------------------------
---------------------------------
---------------------------------


/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:485: RuntimeWarning:

overflow encountered in multiply

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:485: RuntimeWarning:

invalid value encountered in multiply



Warning - optimization restart 3/3 failed
---------------------------------
---------------------------------
---------------------------------
---------------------------------


In [8]:
RMSE_1 = np.zeros((out1.shape[0],3))
RMSE_1[:,0]=out1[:,0,0]
RMSE_1[:,1] = np.mean(out1[:,1,:], axis=1)
RMSE_1[:,2] = np.std(out1[:,1,:], axis=1)/np.sqrt(out1.shape[2])
print(RMSE_1)

[[  0.          85.39648791   8.17734652]
 [  1.          74.17389943   4.40093564]
 [  2.          76.1203637    6.2488749 ]
 [  3.          60.48705567   4.81674052]
 [  4.          62.58564115   3.20107502]
 [  5.          54.29548229   3.12328507]
 [  6.          50.63811532   2.86507395]
 [  7.          49.41290274   3.07886976]
 [  8.          43.21139257   3.13325169]
 [  9.          39.77074093   3.72974198]
 [ 10.          38.80963523   4.20834969]
 [ 11.          32.65379863   4.93287492]
 [ 12.          33.90189294   6.24602094]
 [ 13.          17.05137033   0.66614708]
 [ 14.          14.35007491   0.55129042]
 [ 15.          12.29183196   0.37734687]
 [ 16.          11.45011903   0.27973583]
 [ 17.          10.85045895   0.30083539]
 [ 18.           9.85645448   0.2394253 ]
 [ 19.           9.49109829   0.23797192]
 [ 20.           9.14516582   0.25009281]
 [ 21.           8.71013877   0.24533836]
 [ 22.           7.94610126   0.15660763]
 [ 23.           7.86098197   0.23

The second method trains two GPs, which are each initialised with (different) random starting points. Then new points are added to the design for both GP by seeing which point has the largest difference between the two predictions.

In [10]:
def AddPoint_Method2(m1, m2, Xref):
    mu1,var1 = m1._raw_predict(Xref)
    mu2,var2 = m2._raw_predict(Xref)
    diffs = np.abs(mu2-mu1)
    index = np.argmax(diffs) # take the point with the largest discrepancy
    Xtrain_new1 = np.concatenate((m1.X,Xref[index,:][None,:]),axis=0)
    Xtrain_new2 = np.concatenate((m2.X,Xref[index,:][None,:]),axis=0)
    ytrain_new1 = np.append(m1.Y, branin(Xtrain_new1[-1,:]))[:,None]
    ytrain_new2 = np.append(m2.Y, branin(Xtrain_new2[-1,:]))[:,None]
    Xref_new = np.delete(Xref, index, axis=0)# remove index row
    return(index, Xtrain_new1, ytrain_new1, Xtrain_new2, ytrain_new2, Xref_new)

def AssessMethod2(nMax, nMC):
    RMSE_2 = np.zeros((nMax,3,nMC))
    for mm in range(nMC):
        print('---------------------------------')
        index0_1 = np.random.randint(Xfinegrid.shape[0])
        index0_2 = np.random.randint(Xfinegrid.shape[0])
        
        X0_1 = Xfinegrid[index0_1,:].reshape((1,2))
        X0_2 = Xfinegrid[index0_2,:].reshape((1,2))
        
        Xref = np.delete(Xfinegrid,(index0_1, index0_2), axis=0)
        ytrain1 = branin(X0_1)
        Xtrain1 = X0_1
        ytrain2 = branin(X0_2)
        Xtrain2 = X0_2
       
        k1 = GPy.kern.Matern52(2, ARD=True)#GPy.kern.Bias(2)+GPy.
        m1 = GPy.models.GPRegression(Xtrain1, ytrain1,k1)
        m1.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
        m1.optimize_restarts(3, robust=True, verbose=False)

        
        k2 = GPy.kern.Matern52(2, ARD=True)#GPy.kern.Bias(2)+GPy.
        m2 = GPy.models.GPRegression(Xtrain2, ytrain2,k2)
        m2.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
        m2.optimize_restarts(3, robust=True, verbose=False)

        
        for ii in range(nMax):
            #print('HI')
            index, Xtrain1, ytrain1, Xtrain2, ytrain2, Xref = AddPoint_Method2(m1, m2, Xref)

            k1 = GPy.kern.Matern52(2, ARD=True)#GPy.kern.Bias(2)+GPy.
            m1 = GPy.models.GPRegression(Xtrain1, ytrain1,k1)
            m1.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
            m1.optimize_restarts(5, robust=True, verbose=False)

            k2 = GPy.kern.Matern52(2, ARD=True)#GPy.kern.Bias(2)+GPy.
            m2 = GPy.models.GPRegression(Xtrain2, ytrain2,k2)
            m2.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
            m2.optimize_restarts(5, robust=True, verbose=False)

            #print(m)
            mu1, _ = m1._raw_predict(Xref)
            mu2, _ = m2._raw_predict(Xref)
            
            RMSE_2[ii,2,mm] = np.sqrt(np.mean((mu2 - branin(Xref))**2))
            RMSE_2[ii,1,mm] = np.sqrt(np.mean((mu1 - branin(Xref))**2))
            RMSE_2[ii,0,mm] = ii                           
    return(RMSE_2)

out2 =AssessMethod2(40,10)

---------------------------------


/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:229: RuntimeWarning:

invalid value encountered in true_divide

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:129: RuntimeWarning:

overflow encountered in square

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:130: RuntimeWarning:

invalid value encountered in add

/Users/pmzrdw/anaconda/lib/python3.5/site-packages/paramz/transformations.py:106: RuntimeWarning:

invalid value encountered in greater

/Users/pmzrdw/anaconda/lib/python3.5/site-packages/paramz/transformations.py:111: RuntimeWarning:

invalid value encountered in greater



Warning - optimization restart 1/5 failed
Warning - optimization restart 4/5 failed
Warning - optimization restart 1/5 failed


/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

invalid value encountered in multiply



Warning - optimization restart 1/5 failed
Warning - optimization restart 4/5 failed
Warning - optimization restart 2/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 5/5 failed


/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:158: RuntimeWarning:

overflow encountered in true_divide



Warning - optimization restart 1/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 2/5 failed
Warning - optimization restart 4/5 failed
---------------------------------
---------------------------------
Warning - optimization restart 1/5 failed
Warning - optimization restart 5/5 failed


/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:130: RuntimeWarning:

overflow encountered in multiply

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:130: RuntimeWarning:

overflow encountered in add

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

overflow encountered in multiply



---------------------------------
---------------------------------
Warning - optimization restart 2/5 failed
Warning - optimization restart 3/5 failed
Warning - optimization restart 4/5 failed
Warning - optimization restart 5/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 2/5 failed
---------------------------------
Warning - optimization restart 1/5 failed
Warning - optimization restart 4/5 failed
Warning - optimization restart 2/5 failed
Warning - optimization restart 5/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 2/5 failed
Warning - optimization restart 3/5 failed
Warning - optimization restart 5/5 failed
---------------------------------
Warning - optimization restart 3/5 failed
Warning - optimization restart 5/5 failed
Warning - optimization restart 2/5 failed
Warning - optimization restart 3/5 failed
Warning - optimization restart 1/5 failed
Warning - optimization restart 2/5 failed
------------------------

In [18]:
RMSE_2a = np.zeros((out2.shape[0],3))
RMSE_2a[:,0]=out2[:,0,0]
RMSE_2a[:,1] = np.mean(out2[:,1,:], axis=1)
RMSE_2a[:,2] = np.std(out2[:,1,:], axis=1)/np.sqrt(out2.shape[2])
RMSE_2b = np.zeros((out2.shape[0],3))
RMSE_2b[:,0]=out2[:,0,0]
RMSE_2b[:,1] = np.mean(out2[:,2,:], axis=1)
RMSE_2b[:,2] = np.std(out2[:,2,:], axis=1)/np.sqrt(out2.shape[2])


### Compare with LHC design
I've used more Monte Carlo replicates here as the LHC design seems to give far more variation in the error than the sequential approaches do.


In [19]:
import pyDOE


In [24]:
def Assess_LHC(n, nMC):
    #n =design size
    #nMC = number of Monte Carlo replicates to take
    RMSE_lhc = np.zeros(nMC)
    for ii in range(nMC):
        Xlhc=pyDOE.lhs(2,n,'maximin')
        Xlhc[:,0] = Xlhc[:,0]*15-5.
        Xlhc[:,1] = Xlhc[:,1]*15.
        klhc = GPy.kern.Matern52(2)#GPy.kern.RBF(2)+GPy.kern.Bias(2)
        ylhc = branin(Xlhc)
        mlhc = GPy.models.GPRegression(Xlhc, ylhc,klhc)
        mlhc.likelihood.variance.fix(1e-5)#m.Gaussian_noise.constrain_bounded(0,10)
        
        mlhc.optimize_restarts(3, robust=True, verbose=False)
        mulhc,var1 = mlhc._raw_predict(Xfinegrid)
        RMSE_lhc[ii] = np.sqrt(np.mean((mulhc - branin(Xfinegrid))**2)) 
    return(RMSE_lhc)

#RMSE_lhc=Assess_LHC(61,20)

nvals = np.arange(20,41,2)
RMSE_lhc = np.zeros((nvals.size,3))
for j,ii in enumerate(nvals):
    out = Assess_LHC(ii,30)
    RMSE_lhc[j,1]= np.mean(out)
    RMSE_lhc[j,2] = np.std(out)*1.96/np.sqrt(out.size)
    RMSE_lhc[j,0] =ii
    #print(ii)
    print(j)
    

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:160: RuntimeWarning:

overflow encountered in true_divide

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

invalid value encountered in multiply

/Users/pmzrdw/anaconda/lib/python3.5/site-packages/paramz/transformations.py:106: RuntimeWarning:

invalid value encountered in greater

/Users/pmzrdw/anaconda/lib/python3.5/site-packages/paramz/transformations.py:111: RuntimeWarning:

invalid value encountered in greater

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

overflow encountered in square

/Users/pmzrdw/GPy/GPy/kern/src/stationary.py:482: RuntimeWarning:

overflow encountered in multiply



0
1
2
3
4
5
6
7
8
9
10


In [25]:
err1= go.Scatter(
        x=RMSE_1[:,0]+2,
        y=RMSE_1[:,1],
        mode='markers+line',
        name='Method 1',
        error_y=dict(
            type='data',
            array=RMSE_1[:,2],
            visible=True
        )
        )
err2a= go.Scatter(
        x=RMSE_2a[:,0]+2,
        y=RMSE_2a[:,1],
        mode='markers+line',
        name='Method 2, GP1',
        error_y=dict(
            type='data',
            array=RMSE_2a[:,2],
            visible=True
        )
        )
err2b= go.Scatter(
        x=RMSE_2b[:,0]+2,
        y=RMSE_2b[:,1],
        mode='markers+line',
        name='Method 2, GP2',
        error_y=dict(
            type='data',
            array=RMSE_2b[:,2],
            visible=True
        )
        )

err3=    go.Scatter(
        x=RMSE_lhc[:,0],
        y=RMSE_lhc[:,1],
        mode='markers',
        name='LHC',
        error_y=dict(
            type='data',
            array=RMSE_lhc[:,2],
            visible=True
        )
    )

layout = go.Layout(
    xaxis=dict(
        range=[20, 45]
    ),
    yaxis=dict(
        range=[0, 15]
    )
)

data=[err1, err2a, err2b, err3]
#data=[err3]

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

### Thoughts

I really don't understand this. But it seems to work. Some thoughts:
- is it due to the grid we've used for the set of candidate points?
- is it robust to different GP choices? For the RBF covariance function, I can imagine a single point making a big difference. How does it for Matern? Presumably if we have a decent nugget term (rather than the very small value used here) this method doesn't work?
- I did some experiments learning the nugget size - it still worked but this needs more checking.

I'm keen to think more about this - it seems like a useful method that could be publicised more widely. To generalise it:
- What difference does it make starting from largest sets (ie not just initial designs of size 1)
- Does using more than 2 GPs give any benefit?
- 

How does it compare to MICE?
